In [2]:
import fundamentalanalysis as fa
import financedatabase as fd
import yfinance as yf

import pandas as pd
import datetime as dt
from pathlib import Path
from os.path import exists

In [3]:
pd.set_option('display.max_columns', None)  

In [4]:
api_key = "c3f92edf0e79c46818aabbb8dc76043f"

day1 = '2023-01-31'
month1 = str(day1)[:7]
year1 = str(day1)[:4]

save_hist = Path(f"/home/gdp/hot_box/etfs/data/hist/{year1}{month1}/{day1}/")
if not save_hist.exists():
    save_hist.mkdir(parents=True)

save_profile = Path(f"/home/gdp/hot_box/etfs/data/profile/")
if not save_profile.exists():
    save_profile.mkdir(parents=True)

save_bulk = Path(f"/home/gdp/hot_box/etfs/data/bulk/{year1}{month1}/{day1}/")
if not save_bulk.exists():
    save_bulk.mkdir(parents=True)    

In [5]:
# Show the available companies
companies = fa.available_companies(api_key)
companies = companies.drop_duplicates()
print(companies.shape)

companies.to_csv(save_bulk / "bulk_tickers.csv")

(66946, 5)


In [6]:
etfs = pd.DataFrame(companies[companies['type'] == 'etf'])
etfs = etfs.drop_duplicates()
etfs.to_csv(save_bulk / "etf_tickers.csv")
etfs

,name,price,exchange,exchangeShortName,type
symbol,,,,,
251340.KS,KODEX KOSDAQ150 Inverse,5120.000,KSE,KSC,etf
371460.KS,Mirae Asset Management - Tiger China Electric ...,12895.000,KSE,KSC,etf
233740.KS,KODEX KOSDAQ150 Leverage,7190.000,KSE,KSC,etf
329200.KS,Mirae Asset Management - Tiger Real Estate Inf...,4855.000,KSE,KSC,etf
252670.KS,Samsung Kodex F Kospi 200 Inverse2x ETF,2865.000,KSE,KSC,etf
...,...,...,...,...,...
REC,Emles Real Estate Credit ETF,20.055,BATS,ETF,etf
TCBT.L,VanEck iBoxx EUR Corporates UCITS ETF,16.952,London Stock Exchange,LSE,etf
HEDF.L,WisdomTree Europe Equity UCITS ETF,19.350,London Stock Exchange,LSE,etf


In [15]:
exchange_list = [
    # 'MCX',
    'Other OTC',
    'Toronto Stock Exchange',
    # 'Saudi',
    'New York Stock Exchange Arca',
    'Hamburg',
    'Amsterdam',
    'KSE',
    'Munich',
    'New York Stock Exchange',
    'SES',
    'Taiwan',
    'Milan',
    'BATS',
    'Mexico',
    'London Stock Exchange',
    'Nasdaq Global Select',
    'NASDAQ Capital Market',
    'Swiss Exchange',
    'Paris',
    'NASDAQ Global Select',
    'Frankfurt',
    'NEO',
    'Brussels',
    'NASDAQ Global Market',
    'American Stock Exchange',
    # 'XETRA'
]

In [16]:
etfs[etfs['exchange'].isin(exchange_list)]

,name,price,exchange,exchangeShortName,type
symbol,,,,,
251340.KS,KODEX KOSDAQ150 Inverse,5120.000,KSE,KSC,etf
371460.KS,Mirae Asset Management - Tiger China Electric ...,12895.000,KSE,KSC,etf
233740.KS,KODEX KOSDAQ150 Leverage,7190.000,KSE,KSC,etf
329200.KS,Mirae Asset Management - Tiger Real Estate Inf...,4855.000,KSE,KSC,etf
252670.KS,Samsung Kodex F Kospi 200 Inverse2x ETF,2865.000,KSE,KSC,etf
...,...,...,...,...,...
REC,Emles Real Estate Credit ETF,20.055,BATS,ETF,etf
TCBT.L,VanEck iBoxx EUR Corporates UCITS ETF,16.952,London Stock Exchange,LSE,etf
HEDF.L,WisdomTree Europe Equity UCITS ETF,19.350,London Stock Exchange,LSE,etf


In [7]:
list(set(etfs['exchange']))

['MCX',
 'Other OTC',
 'Toronto Stock Exchange',
 'Saudi',
 'New York Stock Exchange Arca',
 'Hamburg',
 'Amsterdam',
 'KSE',
 'Munich',
 'New York Stock Exchange',
 'SES',
 'Taiwan',
 'Milan',
 'BATS',
 'Mexico',
 'London Stock Exchange',
 'Nasdaq Global Select',
 'NASDAQ Capital Market',
 'Swiss Exchange',
 'Paris',
 'NASDAQ Global Select',
 'Frankfurt',
 'NEO',
 'Brussels',
 'NASDAQ Global Market',
 'American Stock Exchange',
 'XETRA']

In [56]:
def source_profiles(ticker_list):
    good_etf_tickers = []
    bad_etf_tickers = []
    
    try:
        for ticker in ticker_list:
            x = pd.DataFrame(fa.profile(ticker, api_key))
            x.columns = [ticker]
            x.to_csv(save_profile / f"{ticker}_profile.csv")
            good_etf_tickers.append(ticker)

    except Exception as e:
        print(f"X X X ERROR X X X \n [ X ] {ticker} - {e}")
        bad_etf_tickers.append(ticker)
        pass

    return sorted(good_etf_tickers), sorted(bad_etf_tickers)

In [57]:
etf_ticker_list = list(etfs.index)


good_etf_tickers, bad_etf_tickers = source_profiles(etf_ticker_list)

X X X ERROR X X X 
 [ X ] CAPE - Length mismatch: Expected axis has 2 elements, new values have 1 elements


In [60]:
core_selection = fd.select_etfs(exclude_exchanges=True)

tickers = pd.Series(core_selection.keys())
tickers.to_excel(save_bulk / f'core_selection_tickers.xlsx', index=None, header=None)

In [61]:
all_etfs_df = pd.DataFrame.from_dict(fd.select_etfs(), orient='index').reset_index()
all_etfs_df.head()

,index,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
0,AAA,Listed Funds Trust AAF First Pr,AAF First Priority CLO Bond ETF,USD,The investment seeks capital preservation and ...,None,Alternative Access Funds,PCX,us_market,1.001962e+07
1,AAAU,Goldman Sachs Physical Gold ETF,Goldman Sachs Physical Gold ETF,USD,The investment seeks to provide investors with...,None,Exchange Traded Concepts,PCX,us_market,3.317956e+08
2,AADR,AdvisorShares Dorsey Wright,AdvisorShares Dorsey Wright ADR ETF,USD,None,None,None,NMS,us_market,NaN
3,AAXJ,iShares MSCI All Country Asia e,iShares MSCI All Country Asia ex Japan ETF,USD,The investment seeks to track the investment r...,Pacific/Asia ex-Japan Stk,iShares,NMS,us_market,6.588896e+09
4,ABCS,Guggenheim ABC High Dividend ET,None,None,None,None,None,PCX,us_market,NaN


In [62]:
gas = pd.DataFrame.from_dict(fd.search_products(core_selection, 'gas'), orient='index')
gas.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DDG,ProShares Short Oil & Gas,ProShares Short Oil & Gas,USD,The investment seeks daily investment results ...,Trading--Inverse Equity,ProShares,PCX,us_market,3.326665e+06


In [63]:
oil = pd.DataFrame.from_dict(fd.search_products(core_selection, 'oil'), orient='index')
oil.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DBO,Invesco DB Oil Fund,Invesco DB Oil Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,5.195419e+08


In [64]:
dodds_etf_df = oil.append(gas).reset_index().drop_duplicates().sort_values('index')
dodds_etf_df.columns = ['ticker', 'short_name', 'long_name', 'currency', 'summary', 'category', 'family', 'exchange', 'market', 'total_assets']
dodds_etf_df.head()

,ticker,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
0,BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
37,BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07
1,CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
2,DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
3,DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07


In [93]:
x = fd.select_etfs()
x1 = pd.DataFrame.from_dict(x, orient='index')
y = pd.DataFrame.from_dict(fd.search_products(database=x, query='oil', search='summary'), orient='index')
y.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DBO,Invesco DB Oil Fund,Invesco DB Oil Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,5.195419e+08


In [94]:
category_list = [
    # 'Short Government',
    # 'Convertibles',
    # 'Infrastructure',
    # 'Foreign Large Blend',
    # 'Foreign Small/Mid Blend',
    # 'Miscellaneous Sector',
    # 'Foreign Large Value',
    # 'Muni California Long',
    # 'Mid-Cap Growth',
    # 'Financial',
    # 'Short-Term Bond',
    # 'India Equity',
    # 'Foreign Large Growth',
    'Equity Energy',
    # 'Muni Minnesota',
    # 'Latin America Stock',
    # 'Mid-Cap Blend',
    # 'Long-Short Equity',
    # 'Miscellaneous Region',
    # 'Allocation--85%+ Equity',
    # 'Small Blend',
    # 'China Region',
    # 'Intermediate Government',
    # 'Single Currency',
    # 'Multialternative',
    # 'Large Growth',
    'Energy Limited Partnership',
    # 'High Yield Bond',
    # 'Consumer Defensive',
    # 'Volatility',
    'Trading--Inverse Commodities',
    # 'Market Neutral',
    # 'Diversified Pacific/Asia',
    # 'Communications',
    # 'Health',
    # 'Trading--Leveraged Debt',
    # 'Foreign Small/Mid Value',
    # 'Pacific/Asia ex-Japan Stk',
    # 'Muni National Interm',
    # 'Managed Futures',
    # 'Muni National Short',
    # 'Bear Market',
    'Natural Resources',
    # None,
    # 'Consumer Cyclical',
    # 'Small Growth',
    # 'High Yield Muni',
    'Trading--Leveraged Commodities',
    # 'Long-Short Credit',
    # 'Ultrashort Bond',
    # 'Option Writing',
    # 'Equity Precious Metals',
    # 'Allocation--15% to 30% Equity',
    'Utilities',
    # 'Preferred Stock',
    # 'Long Government',
    # 'Corporate Bond',
    'Commodities Broad Basket',
    # 'Multisector Bond',
    # 'Small Value',
    # 'World Bond',
    # 'Bank Loan',
    # 'Allocation--70% to 85% Equity',
    # 'Japan Stock',
    # 'Tactical Allocation',
    # 'Multicurrency',
    # 'Nontraditional Bond',
    # 'Long-Term Bond',
    # 'Large Blend',
    # 'World Allocation',
    # 'Foreign Small/Mid Growth',
    # 'Large Value',
    'Industrials',
    # 'Global Real Estate',
    # 'Muni National Long',
    # 'Europe Stock',
    # 'World Stock',
    # 'Trading--Inverse Equity',
    # 'Trading--Leveraged Equity',
    # 'Diversified Emerging Mkts',
    # 'Inflation-Protected Bond',
    # 'Allocation--50% to 70% Equity',
    # 'Trading--Inverse Debt',
    # 'Technology',
    # 'Allocation--30% to 50% Equity',
    # 'Intermediate-Term Bond',
    # 'Trading--Miscellaneous',
    # 'Real Estate',
    # 'Mid-Cap Value',
    # 'Emerging Markets Bond',
    # 'Muni New York Intermediate',
    # 'Emerging-Markets Local-Currency Bond'
 ]

In [95]:
for c in category_list:
    z = pd.DataFrame.from_dict(fd.search_products(database=x, query=c, search='summary'), orient='index')
    y = y.append(z)

In [96]:
y = pd.DataFrame(y).reset_index().sort_values('index')
yy = y.drop_duplicates(subset=['index'])
yy = yy.rename(columns={"index": "ticker"})
# yy.to_csv(save_bulk)

In [98]:
dodds_etf_df = pd.DataFrame(dodds_etf_df.append(yy).drop_duplicates(subset=['ticker'])).sort_values('ticker')
dodds_etf_df

,ticker,short_name,long_name,currency,summary,category,family,exchange,market,total_assets,level_0
0,ARMR,Exchange Listed Funds Trust Arm,Armor US Equity Index ETF,USD,The investment seeks to provide investment res...,Large Blend,Exchange Traded Concepts,PCX,us_market,6.117832e+06,66.0
0,BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08,NaN
37,BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07,NaN
3,CHII,Global X MSCI China Industrials,Global X MSCI China Industrials ETF,USD,The investment seeks to provide investment res...,China Region,Global X Funds,PCX,us_market,2.409662e+06,67.0
4,CHIU,Global X Funds Global X MSCI Ch,Global X MSCI China Utilities ETF,USD,The investment seeks to provide investment res...,None,Global X Funds,PCX,us_market,1.706715e+06,48.0
...,...,...,...,...,...,...,...,...,...,...,...
74,VPU,Vanguard Utilities ETF,Vanguard Utilities Index Fund ETF Shares,USD,The investment seeks to track the performance ...,Utilities,Vanguard,PCX,us_market,6.270528e+09,64.0
34,XES,SPDR Series Trust SPDR S&P Oil,SPDR S&P Oil & Gas Equipment & Services ETF,USD,The investment seeks to provide investment res...,Equity Energy,SPDR State Street Global Advisors,PCX,us_market,1.547576e+08,NaN
35,XLE,SPDR Select Sector Fund - Energ,Energy Select Sector SPDR Fund,USD,The investment seeks to provide investment res...,Equity Energy,SPDR State Street Global Advisors,PCX,us_market,2.308107e+10,NaN
73,XLU,SPDR Select Sector Fund - Utili,Utilities Select Sector SPDR Fund,USD,The investment seeks to provide investment res...,Utilities,SPDR State Street Global Advisors,PCX,us_market,1.236013e+10,NaN


In [102]:
dodds_etf_df.to_csv(save_bulk / f"dodds_etf_df.csv")

In [103]:
dodds_etf_ticker_list = list(dodds_etf_df["ticker"])
len(dodds_etf_ticker_list)

86

In [105]:
dodds_etf_hist_df = yf.download(dodds_etf_ticker_list, period='1y', rounding=True)['Adj Close']

[*********************100%***********************]  86 of 86 completed

1 Failed download:
- FRAK: No data found for this date range, symbol may be delisted


In [107]:
dodds_etf_hist_df.dropna(axis=1)

,ARMR,BNO,BOIL,CHII,CHIU,CRAK,DBC,DBE,DBO,DIG,DRIP,DUG,DUSL,EDOW,EINC,EMLP,ERTH,ERX,ERY,ETHO,EXI,FCG,FIDU,FTRI,FTXN,FUE,FUTY,FXR,FXU,GAZ,GNR,GUNR,GUSH,HAP,IDU,IEO,IEZ,IGE,IYE,IYJ,JXI,KOLD,MUNI,NANR,NRGD,NRGU,OIH,OILK,PDBC,PSCE,PSCI,PSCU,PUI,PXE,PXJ,RGI,RLY,RYU,SCO,SDP,SIJ,UCO,UGA,UGAZF,UMI,UNG,UNL,UPW,USL,USO,USOI,UTES,UTSL,UXI,VIS,VPU,XES,XLE,XLU,XOP
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-02-01,24.68,24.19,46.25,16.15,17.54,27.97,22.37,19.35,15.03,27.66,38.20,30.24,41.59,30.84,57.31,24.80,59.26,43.22,71.20,58.11,115.43,19.52,52.53,13.75,22.32,10.96,43.29,57.75,31.17,25.77,55.17,40.55,115.34,47.83,82.04,70.66,15.95,34.57,34.94,107.78,60.72,114.44,54.05,47.36,1830.00,299.56,232.78,40.03,13.44,7.92,89.41,63.03,33.49,22.74,3.84,184.84,26.52,108.53,48.40,13.43,14.68,28.06,47.53,10.50,30.51,17.10,15.15,66.80,31.63,62.53,75.47,44.01,34.04,30.06,189.94,144.68,60.89,65.48,66.34,108.32
2022-02-02,24.83,24.22,56.02,16.13,17.53,28.12,22.46,19.55,15.05,27.82,37.60,30.10,42.29,31.01,57.83,25.00,58.92,43.55,70.80,58.23,116.43,19.76,52.78,13.85,22.47,10.99,43.84,57.84,31.56,28.51,55.47,40.82,116.50,48.08,83.09,71.44,15.73,34.72,35.05,107.30,61.34,85.24,54.14,47.62,1795.00,306.10,230.30,40.06,13.52,7.92,88.77,62.55,33.86,22.98,3.78,185.86,26.71,110.18,48.30,13.12,14.87,28.12,48.01,13.50,30.73,18.98,16.08,68.47,31.72,62.52,75.61,44.54,35.49,29.68,190.73,146.83,59.97,65.69,67.32,109.05
2022-02-03,24.62,24.60,45.56,16.05,17.44,27.80,22.51,19.65,15.25,27.23,38.70,30.77,40.27,30.61,57.35,24.86,57.84,42.73,71.99,57.01,114.12,19.51,51.82,13.76,22.21,10.80,43.66,56.88,31.36,25.97,54.88,40.44,113.23,47.46,82.46,70.54,15.47,34.31,34.63,105.14,61.02,99.76,54.22,47.14,1840.00,297.79,227.40,40.67,13.51,7.79,87.38,62.13,33.60,22.76,3.74,182.59,26.51,109.92,46.85,13.22,15.51,28.97,48.42,9.99,30.48,17.24,15.44,67.94,32.22,63.51,76.04,44.27,34.97,28.40,187.35,145.90,58.89,65.07,67.02,107.46
2022-02-04,24.70,25.20,38.16,16.12,17.56,28.14,22.69,19.80,15.48,28.05,37.40,29.79,39.00,30.57,57.83,24.79,58.47,44.04,69.70,57.19,113.50,19.81,51.38,13.83,22.46,10.91,43.29,56.40,31.07,23.81,55.38,40.80,116.78,47.73,81.93,71.57,15.72,34.60,35.15,104.47,60.58,117.36,54.18,47.64,1745.00,314.06,232.03,41.46,13.62,7.96,85.89,62.24,33.39,23.11,3.84,180.70,26.67,108.95,45.00,13.41,15.69,30.12,49.10,7.45,30.74,15.84,14.52,66.99,32.83,64.91,76.04,44.00,34.21,28.11,185.70,144.67,60.37,66.08,66.49,109.12
2022-02-07,24.79,25.07,33.85,16.14,17.67,28.50,22.74,19.74,15.49,28.69,37.80,29.13,39.18,30.51,57.82,24.83,57.98,45.21,68.00,57.13,113.71,19.61,51.39,14.06,22.47,11.10,43.20,56.32,31.03,22.55,55.96,41.32,115.61,48.27,81.75,71.74,15.93,34.92,35.58,104.09,60.24,129.04,54.13,48.10,1675.00,326.98,234.21,41.38,13.65,7.80,86.16,62.16,33.43,23.05,3.84,180.75,26.74,108.84,45.15,13.50,15.86,30.03,49.40,6.31,30.82,14.99,13.82,66.45,32.77,64.68,76.60,43.80,34.04,27.80,185.91,144.48,59.93,66.93,66.39,108.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-25,21.18,28.45,8.52,14.01,15.73,33.40,24.94,22.41,15.36,44.65,12.89,11.67,33.47,29.72,63.79,27.33,51.42,70.27,27.33,53.02,111.92,25.20,52.37,14.78,29.27,11.99,44.58,54.98,33.08,15.70,61.25,46.12,157.34,52.01,84.45,96.46,22.58,42.61,47.63,100.45,60.24,48.25,52.42,58.05,277.22,549.95,324.85,45.14,15.02,10.73,94.21,57.50,33.70,30.98,5.46,186.04,28.83,114.69,23.29,11.39,14.59,30.55,62.57,3.97,34.81,9.81,13.65,65.96,35.73,70.51,83.83,45.77,30.83,24.02,189.13,149.92,86.43,90.26,68.66,142.04
2023-01-26,21.46,28.73,8.12,14.12,15.59,33.92,25.05,22.53,15.42,47.46,12.28,10.95,34.00,29.90,64.39,27.53,52.08,74.69,25.60,53.45,112.40,25.64,52.67,14.92,30.16,12.01,44.72,55.47,33.18,15.38,62.08,46.61,165.10,52.71,84.65,98.78,22.93,

In [108]:
for i in dodds_etf_ticker_list:
    print(i)

ARMR
BNO
BOIL
CHII
CHIU
CRAK
DBC
DBE
DBO
DDG
DIG
DRIP
DUG
DUSL
EDOW
EINC
EMLP
ERTH
ERX
ERY
ETHO
EXI
FCG
FIDU
FRAK
FTRI
FTXN
FUE
FUTY
FXR
FXU
GAZ
GNR
GUNR
GUSH
HAP
IDU
IEO
IEZ
IGE
IMLP
IYE
IYJ
JHMI
JHMU
JXI
KOLD
MUNI
NANR
NRGD
NRGU
OIH
OILK
PDBC
PSCE
PSCI
PSCU
PUI
PXE
PXJ
PZD
RGI
RLY
RYU
SCO
SDP
SIJ
UCO
UGA
UGAZF
UMI
UNG
UNL
UPW
USL
USO
USOI
UTES
UTSL
UXI
VIS
VPU
XES
XLE
XLU
XOP
